In [142]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
import string

### Reading the train and test csv files

In [143]:
df_train_data=pd.read_csv("0000000000002747_training_twitter_x_y_train.csv")
df_test_data=pd.read_csv("0000000000002747_test_twitter_x_test.csv")

In [144]:
df_train_data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


### extracting X and Y from the train data

In [145]:
df_train_data=df_train_data[['airline_sentiment','text']]
df_train_data.head()

,airline_sentiment,text
0,negative,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,@SouthwestAir seeing your workers time in and ...
2,positive,@united Flew ORD to Miami and back and had gr...
3,negative,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,negative,@united so our flight into ORD was delayed bec...


## Modifying our data to be fitted

### neagtive - 2
### positive - 1
### neutral -0

In [146]:
def transform_Y(df):
    df.loc[df.airline_sentiment=='negative',['airline_sentiment']]=2
    df.loc[df.airline_sentiment=='positive',['airline_sentiment']]=1
    df.loc[df.airline_sentiment=='neutral',['airline_sentiment']]=0
    return df

In [147]:
df_train_data=transform_Y(df_train_data)
df_train_data.head()

,airline_sentiment,text
0,2,"@SouthwestAir I am scheduled for the morning, ..."
1,1,@SouthwestAir seeing your workers time in and ...
2,1,@united Flew ORD to Miami and back and had gr...
3,2,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,2,@united so our flight into ORD was delayed bec...


### Splitting the train data df into train and test data with a 7:3 ratio

In [148]:
number=df_train_data.shape[0]
number

10980

In [149]:
val=int((number/10)*7)
val

7686

In [150]:
df_train=df_train_data[0:val]
df_train.shape

(7686, 2)

In [151]:
df_test=df_train_data[val:]
df_test.shape

(3294, 2)

In [152]:
X_train=df_train['text']
Y_train=df_train['airline_sentiment']
X_test=df_test['text']
Y_test=df_test['airline_sentiment']

In [153]:
X_train.head()

0    @SouthwestAir I am scheduled for the morning, ...
1    @SouthwestAir seeing your workers time in and ...
2    @united Flew ORD to Miami and back and  had gr...
3       @SouthwestAir @dultch97 that's horse radish 😤🐴
4    @united so our flight into ORD was delayed bec...
Name: text, dtype: object

In [154]:
Y_train.head()

0    2
1    1
2    1
3    2
4    2
Name: airline_sentiment, dtype: object

### Converting y train and test into a 1D array

In [155]:
y_train=list(Y_train)
y_train

[2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 0,
 1,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 2,
 0,
 1,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 1,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,


In [156]:
y_test=list(Y_test)
y_test

[2,
 0,
 1,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 0,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 1,
 2,
 0,
 2,
 1,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 1,
 2,
 2,
 0,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 0,
 0,
 0,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,


### Getting all the stop words

In [157]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))  
print(stop_words)

{'me', 'a', 'hers', 'to', 'same', 'myself', 'did', 'their', 'out', 'ain', 'yourselves', 'does', "you're", 've', 'm', 'will', 'once', 'now', 'those', "you'd", 'has', 'which', 'or', 'herself', "shan't", 'are', 'theirs', 'shouldn', "don't", 'won', 'couldn', 'again', 'all', 't', 'any', 'd', 'aren', 'why', 'ma', 'not', "it's", 'doesn', 'he', 'both', 'isn', 'for', "wasn't", 'where', 'with', 'too', "shouldn't", "she's", 'its', 'at', 'no', 'mustn', 'during', 'than', 'weren', 'we', 'been', 'i', "won't", 'after', 'hadn', 'before', 'in', 'nor', 'll', 'between', "wouldn't", 'having', 'most', 'into', "mustn't", 'because', 'was', 'what', 'yours', 'off', 'my', 'under', 'below', 'on', 'over', 'be', 'and', 'mightn', 'about', 'of', 'an', 'until', 's', 'how', 'wouldn', 'her', 'they', "doesn't", 'very', 'wasn', 'that', 'his', 'you', 'themselves', "couldn't", "weren't", 'himself', 'had', "haven't", 'them', 'being', 'more', 'while', 'do', "didn't", 'yourself', "isn't", 'such', 'above', 'am', 'o', 'up', 'the

In [158]:
stops=set(stop_words)
punctuations=list(string.punctuation)
stops.update(punctuations)
stops

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

### Function to clean and convert X into a set of strings

In [159]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer=WordNetLemmatizer()

In [160]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    if tag.startswith('V'):
        return wordnet.VERB
    if tag.startswith('N'):
        return wordnet.NOUN
    if tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [161]:
def X_transform(arr,stop_words):
    doc=[]
    s=[]
    for a in arr:
            doc.append(word_tokenize(a))
    for sen in doc:
        string=''
        for tok in sen:
            if (tok not in stop_words) and (len(tok)>2) and (tok.isalpha()):
                if(tok.lower()):
                    pos=pos_tag([tok])
                    clean_word=lemmatizer.lemmatize(tok,pos=get_simple_pos(pos[0][1]))
                    if(string!=''):
                        string=string+" "+clean_word
                    else:
                        string=clean_word
        if string == '':
            string = " "
        s.append(string)
    return s

In [162]:
train_arr=np.array(X_train) 
train_set=X_transform(train_arr,stops)
train_set

['SouthwestAir schedule morning day fact sure even flight one Cancelled Flightled',
 'SouthwestAir see worker time time go beyond love fly guy Thank',
 'united Flew ORD Miami back great crew service leg THANKS',
 'SouthwestAir horse radish',
 'united flight ORD delayed Air Force One last flight SBN min land',
 'united Why load fly sardine knew pilot hour Late Flight incompetent beyond belief',
 'JetBlue stock response Delays frustrate poor cust serv amp told ppl wait amp come back',
 'JetBlue That nice Hoping rack enough mile take trip Seattle enjoy perfect latte city coffee',
 'united frankly bad customer service ever Problems happen deal defines company Never United',
 'SouthwestAir yeah haha Never one expensive much fun destinationdragons',
 'SouthwestAir DCA flight almost full people screw Cancelled Flightation united USAirways Cancelled Flight',
 'JetBlue easy way get ticket receipt Can get one check get one online Thanks',
 'USAirways love change lounge cheese veggie olive additi

In [163]:
test_arr=np.array(X_test)
test_set=X_transform(test_arr,stops)
test_set

['USAirways move tarmac delayed unknown reason keep post',
 'SouthwestAir show luv Its degree snow Imdetermined DestinationDragons http',
 'VirginAmerica thanks gate check baggage full flight give early boarding sweet',
 'united How know bag You knew hour ago You kill Worst service Ever',
 'JetBlue yes hurt feeling walk MINT fella dream',
 'united flight',
 'JetBlue Just info',
 'united Why force check rollaboard Plenty room overhead next seat http',
 'SouthwestAir hold min Anyone connection another airline get CHI Columbus EWR',
 'SouthwestAir Finally get home BNASnow',
 'united bag leave Washington yet That would convenient see left day ago UnitedAirlines lostluggage',
 'united fact treat disrespectfully lie member airline claim strive best',
 'USAirways lack customer service shin need step get lose baggage delta return',
 'united wait bag IAH They freak people flight wait bag Fix',
 'AmericanAir Thanks',
 'SouthwestAir please tell today flight Chicago DIA Cancelled Flightled Thanks'

## Calling count vectorizer to get the required form of clean data

In [276]:
count_vec=CountVectorizer(max_features=2000,ngram_range=(0,3),max_df=0.8,min_df=0.1)
a=count_vec.fit_transform(train_set)
a.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [277]:
count_vec.get_feature_names()

['able',
 'able get',
 'absolute',
 'absolutely',
 'absurd',
 'accept',
 'acceptable',
 'access',
 'accommodate',
 'account',
 'act',
 'actual',
 'actually',
 'add',
 'additional',
 'address',
 'admiral',
 'admiral club',
 'advantage',
 'advise',
 'advisory',
 'after',
 'afternoon',
 'again',
 'age',
 'agent',
 'agent phone',
 'agent say',
 'ago',
 'ahead',
 'air',
 'aircraft',
 'airline',
 'airline cancelled',
 'airline ever',
 'airlines',
 'airplane',
 'airport',
 'airway',
 'airways',
 'all',
 'allow',
 'almost',
 'almost hour',
 'alone',
 'along',
 'already',
 'also',
 'alternate',
 'although',
 'always',
 'amaze',
 'america',
 'american',
 'american airlines',
 'americanair',
 'americanair already',
 'americanair bad',
 'americanair call',
 'americanair can',
 'americanair cancelled',
 'americanair cancelled flightled',
 'americanair could',
 'americanair delay',
 'americanair flight',
 'americanair flight cancelled',
 'americanair flight delayed',
 'americanair get',
 'americanai

In [278]:
b=count_vec.transform(test_set)
b.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [279]:
train_X=a.todense()
train_X

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [167]:
train_X.shape

(7686, 500)

In [280]:
test_X=b.todense()
test_X

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### We got our train_X,train_Y,test_X,test_Y and now using SVM classifier on the data

In [170]:
from sklearn.svm import SVC

In [171]:
train_X.shape

(7686, 500)

In [172]:
y_train

[2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 0,
 1,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 2,
 0,
 1,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 1,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,


In [290]:
svc=SVC(kernel='linear',gamma=0.0001)
svc.fit(train_X,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [291]:
svc.score(train_X,y_train)

0.9099661722612542

In [292]:
svc.score(test_X,y_test)

0.7522768670309654

### Similarily appplying the model on the test data of our train file

In [176]:
df_test_X=df_test_data['text']
df_test_X.head()

0    @AmericanAir In car gng to DFW. Pulled over 1h...
1    @AmericanAir after all, the plane didn’t land ...
2    @SouthwestAir can't believe how many paying cu...
3    @USAirways I can legitimately say that I would...
4    @AmericanAir still no response from AA. great ...
Name: text, dtype: object

In [177]:
test_test_arr=np.array(df_test_X) 
test_test_set=X_transform(test_test_arr,stops)
test_test_set

['AmericanAir car gng DFW Pulled ago icy road since reach arpt Wat',
 'AmericanAir plane land identical bad condition GRK accord METARs',
 'SouthwestAir believe many pay customer left high dry reason flight Cancelled Flightlations Monday BDL Wow',
 'USAirways legitimately say would rather driven cross country flown Airways',
 'AmericanAir still response great job guy',
 'united developer fly tmrw morn min layover earlier flight layover move',
 'USAirways hello Anyone',
 'USAirways husainhaqqani Husain shld protest well one party member Rehman Malik delayed PIA flight',
 'USAirways likely flightaware say plane still Durango depart',
 'AmericanAir even give option Just say line busy Plz try Late Flightr',
 'united announcement pre boarding address mobility disability require travel lot stuff preboard',
 'USAirways really embarrass ask complimentary detailed http amp argue',
 'SouthwestAir passport time trip Could still fly photo thingsishouldknow ifeeldumb',
 'AmericanAir delayed bag fri

## Now we got our model and now we can predict the sentiment for our test file

In [293]:
c=count_vec.fit_transform(test_test_set)
test_x=c.todense()
test_x

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [294]:
y_pred=svc.predict(test_x)
y_pred

array([0, 0, 0, ..., 2, 2, 2])

In [295]:
data=[]
for a in y_pred:
    if a==0:
        data.append('neutral')
    if a==1:
        data.append('positive')
    else:
        data.append('negative')
data

['neutral',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',
 'positive',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'negative',
 'positive',
 'negative',

In [316]:
df=pd.DataFrame(data,columns=['predictions'])
df.reset_index(drop=True,inplace=True)
df.head()

,predictions
0,neutral
1,negative
2,neutral
3,negative
4,neutral


In [317]:
df.to_csv('predictions.csv',index=False)